# DreamJobber

**Tech Edition**

---

**Process**
1. Clean text
2. Bag of Words
3. LDA model (Latent Dirichlet allocation)
4. Fine tune LDA model
5. Define Topics from LDA model
6. Create df of document probabilities
6. Classification model

---

**Import Necessary Libraries**

In [32]:
import pandas as pd
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
import nltk
from functions import *
import pickle

#lda model evaluatoin with coherence
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel
from gensim.corpora.dictionary import Dictionary

from sklearn.neighbors import NearestNeighbors

In [33]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/admin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

---

**Load Data**

In [34]:
df_1 = pd.read_json('data/dice_jobs_1.json', lines=True)
df_2 = pd.read_json('data/dice_jobs_2.json', lines=True)
df_3 = pd.read_json('data/dice_jobs_3.json', lines=True)
df_4 = pd.read_json('data/dice_jobs_4.json', lines=True)
df_5 = pd.read_json('data/dice_jobs_5.json', lines=True)
df_6 = pd.read_json('data/dice_jobs_6.json', lines=True)

In [35]:
#concat into one df
df = pd.concat([df_1, df_2, df_3, df_4, df_5, df_6], ignore_index=True, sort=True)

In [36]:
df.head()

,job_description,job_title
0,NaN,UI Lead/Architect
1,NaN,Web Application Architect
2,NaN,Senior DataStage Developer
3,NaN,Hadoop Administrator
4,NaN,UX Visual Designer


In [37]:
#check for missing values
df.isna().sum()

job_description    6524
job_title          5484
dtype: int64

In [38]:
#looks like there are rows that have no job description
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27016 entries, 0 to 27015
Data columns (total 2 columns):
job_description    20492 non-null object
job_title          21532 non-null object
dtypes: object(2)
memory usage: 422.2+ KB


In [39]:
#drop rows with no job descriptions
df = df.dropna()

In [40]:
#sanity check, looks good
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20492 entries, 9 to 27015
Data columns (total 2 columns):
job_description    20492 non-null object
job_title          20492 non-null object
dtypes: object(2)
memory usage: 480.3+ KB


In [41]:
df.head()

,job_description,job_title
9,[2+ years of experience developing Java / J2EE...,Java Developer (Sign-On BONUS!)
10,"[Passion for technology and learning, a natura...","iOS Developer - Mobile Rate: Open, Duration: 1..."
17,[We enjoy approved IT vendor status with sever...,"EUC Engineer, Rate-Open, Duration: 18 Months"
18,[We enjoy approved IT vendor status with sever...,"Software Developer - RPG, Rate-Open, Duration:..."
19,[SME in Linux Operating system with Strong Vir...,Sr. Linux Consultant with Weblogic exp


In [42]:
#need to remove brackets from job_description
df['job_description'] = df['job_description'].map(remove_brackets)

In [43]:
#remove '\\n' and replace with ','
df['job_description'] = df['job_description'].map(lambda x: x.replace('\\n', ','))

In [44]:
#lowercase text before applying stopwords
df['job_description'] = df['job_description'].map(lambda x: x.lower())

In [45]:
#lowercase job_title text before cleaning
df['job_title'] = df['job_title'].map(lambda x: x.lower())

In [46]:
df = df.reset_index(drop=True)
df.head()

,job_description,job_title
0,'2+ years of experience developing java / j2ee...,java developer (sign-on bonus!)
1,"'passion for technology and learning, a natura...","ios developer - mobile rate: open, duration: 1..."
2,'we enjoy approved it vendor status with sever...,"euc engineer, rate-open, duration: 18 months"
3,'we enjoy approved it vendor status with sever...,"software developer - rpg, rate-open, duration:..."
4,'sme in linux operating system with strong vir...,sr. linux consultant with weblogic exp


In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20492 entries, 0 to 20491
Data columns (total 2 columns):
job_description    20492 non-null object
job_title          20492 non-null object
dtypes: object(2)
memory usage: 320.3+ KB


In [48]:
df = df.drop_duplicates()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18717 entries, 0 to 20491
Data columns (total 2 columns):
job_description    18717 non-null object
job_title          18717 non-null object
dtypes: object(2)
memory usage: 438.7+ KB


In [ ]:
#def get_first_title(title):
    # keep "co-founder, co-ceo, etc"
   # title = re.sub(r"[Cc]o[\-\ ]","", title)
    #split_titles = re.split(r"\,|\:|\(|and", title)
    #return split_titles[0].strip()

In [ ]:
#df['job_title1'] = df['job_title'].map(get_first_title)

In [ ]:
df.tail()

In [ ]:
def remove_stop_words(text):
    
    """Remove stopwords from job titles"""
    
    jobtitle_stopwords = ['a',
                          'bonus',
                          'ca',
                          'duration',
                          'direct',
                          'exclusive',
                          'fast',
                          'for',
                          'full-time'
                          'growing',
                          'hire',
                          'implementation',
                          'month',
                          'need',
                          'open',
                          'openings',
                          'rate',
                          'startup',
                          'tx',
                          'workday',
                          'year']
    
    result = []
    
    for word in text.split():
        
        if word not in jobtitle_stopwords:
            result.append(word)
    return result


In [ ]:
df.iloc[20391]

In [ ]:
#remove stop words from job titles
df['job_title1'] = df['job_title1'].map(remove_stop_words)

In [ ]:
df.iloc[20391]

In [ ]:
remove_stop_words('change manager - workday implementation')

In [ ]:
#!pip install inflect

In [ ]:

#nltk.download('averaged_perceptron_tagger')
  

In [18]:
import argparse
import re
import nltk
import sys
import inflect

#Revmoe Punctuations that definitely won't contribute
def remove_punctuation(s):
    #Phase 1, handle punctuation.
    # Directly removable
    #  '/', ',', '\', '|',
    for char in ['/', '//', '|', '?', '(', ')', '{', '}', '[', ']', ',', '.', '-']:
        s = s.replace(char, " ")
    s.strip()
    #remove those consists of only punctuation and number
    s = filter(lambda x: not re.match("[0-9~!@#$%^&*()_\-+{}\":;\']+", x), s.split())
    return ' '.join(s)

In [ ]:
#Read in a file, create a list of string
#def readfile(filelocation):
 #   with open(filelocation) as f:
  #      l = f.readlines()
   #     return [i.strip() for i in l]


In [19]:
def removePunctuation(jobtitle_list):
    return [remove_punctuation(s) for s in jobtitle_list]


In [20]:
job_list = removePunctuation(df['job_title'])

In [21]:
#Word Count
def wordCount(jobtitle_list):
    wordcount = {}
    for jobtitle in jobtitle_list:
        for word in jobtitle.split():
            if word not in wordcount:
                wordcount[word] = 1
            else:
                wordcount[word] += 1
    return wordcount

In [22]:
word_count = wordCount(job_list)

In [ ]:
    jobtitle_stopwords = ['a',
                          'absence',
                          'accepted',
                          'advantage',
                          'allstate',
                          'alteryx',
                          'ana',
                          'and',
                          'angeles',
                          'arher',
                          'apigee',
                          'applications',
                          'at',
                          'atlanta',
                          'austin',
                          'availability',
                          'az',
                          'az!',
                          'bare',
                          'bartender',
                          'bay!!',
                          'beach',
                          'beautiful',
                          'bench',
                          'benefi',
                          'benefits',
                          'benefit!',
                          'bilingual',
                          'bloomington',
                          'bonus',
                          'bonus!',
                          'boston',
                          'brentwood',
                          'briefing',
                          'brooklyn',
                          'ca',
                          'call',
                          'california',
                          'campus',
                          'candidates',
                          'career',
                          'certification',
                          'charlotte',
                          'charleston',
                          'chesterbrook',
                          'chicago',
                          'china',
                          'chef',
                          'chester',
                          'chief',
                          'cincinnati',
                          'cisco',
                          'city',
                          'clearance',
                          'cleared',
                          'click',
                          'cloverleaf',
                          'co',
                          'colorado',
                          'columbus',
                          'come',
                          'commissioning',
                          'commissions',
                          'contact',
                          'contractor',
                          'dallas',
                          'date',
                          'day',
                          'days',
                          'dayforce',
                          'degree',
                          'denver',
                          'detroit',
                          'diego',
                          'diligence',
                          'diversity',
                          'dodgeville',
                          'dover',
                          'due',
                          'dulles',
                          'duration',
                          'durham',
                          'direct',
                          'dirext',
                          'east',
                          'education',
                          'email',
                          'employment',
                          'english',
                          'entry',
                          'enrollments',
                          'espresso',
                          'exclusive',
                          'f1',
                          'fast',
                          'fico',
                          'fixed',
                          'for',
                          'forms',
                          'fort',
                          'friday',
                          'fte',
                          'full-time',
                          'full',
                          'fulltime',
                          'fully',
                          'ga',
                          'german',
                          'graduates',
                          'great',
                          'groovy',
                          'growing',
                          "hl7': 2",
                          'half',
                          'heights',
                          'high',
                          'highly',
                          'higher',
                          'hire',
                          'hiring',
                          'hour',
                          'hours',
                          'houston',
                          'in',
                          'implementation',
                          'immediate',
                          'immediately!',
                          'incentive',
                          'intern',
                          'internship',
                          'interview',
                          'interviewing',
                          'income',
                          'invoice',
                          'irving',
                          'is',  
                          'j2ee',
                          'japanese',
                          'job',
                          'join',
                          'jr',
                          'junior',
                          'kennesaw',
                          'ladera',
                          'las',
                          'lending',
                          'life',
                          'local',
                          'logging',
                          'long',
                          'looking',
                          'los',
                          'loyalty',
                          'louis',
                          'mandarin',
                          'medicare',
                          'memphis',
                          'mi',
                          'miami',
                          'mid',
                          'midnight',
                          'mn',
                          'mo',
                          'monday',
                          'month',
                          'months',
                          'montnths',
                          'montreal',
                          'moving',
                          'multiple',
                          'must',
                          'nashville',
                          'native',
                          'nc',
                          'need',
                          'needed',
                          'new!',
                          'new',
                          'nh',
                          'nightshift',
                          'north',
                          'nj',
                          'ny',
                          'nyc',
                          'of',
                          'oh',
                          'ok',
                          'okc',
                          'on',
                          'oklahoma',
                          'onbase',
                          'only',
                          'only!',
                          'onsite',
                          'open',
                          'opening',
                          'openings',
                          'opportunity',
                          'option',
                          'openings!',
                          'opportunity',
                          'opportunities',
                          'or',
                          'orlando',
                          'our',
                          'overnight',
                          'owner',
                          'paid',
                          'partial',
                          'passion!',
                          'pay',
                          'payroll',
                          'per',
                          'phd',
                          'philadelphia',
                          'phoenix',
                          'phone',
                          'peoplesoft',
                          'portuguese',
                          'perm',
                          'permanent',
                          'positions',
                          'practice',
                          'prelude',
                          'preferred',
                          'prepayment',
                          'profit',
                          'promotion',
                          'quickly!',
                          'raleigh',
                          'rate',
                          'rate:',
                          'recruiter',
                          'recruiting',
                          'recruitment',
                          'redmond',
                          'relocate',
                          'relocation',
                          'remote!',
                          'remote**',
                          'residential',
                          'required',
                          'required:',
                          'required**',
                          'requirements',
                          'richmond',
                          'robert',
                          'role',
                          'salesforce',
                          'san',
                          'santa',
                          'sap',
                          'sc',
                          'seattle',
                          'selection',
                          'shanghai',
                          'sharing',
                          'shift',
                          'sign',
                          'sitecore',
                          'snowflake',
                          'southern',
                          'sox',
                          'spring',
                          'sr',
                          'startup',
                          'st',
                          'sterile',
                          'sterling',
                          'student',
                          'subcontract',
                          'talend',
                          'tampa',
                          'temp',
                          'temporary',
                          'term',
                          'texas',
                          'territory',
                          'transition',
                          'travel',
                          'that',
                          'the',
                          'this',
                          'time',
                          'tn',
                          'travelers',
                          'tri',
                          'ts',
                          'tx',
                          'university',
                          'usc',
                          'va',
                          'valley',
                          'vegas',
                          'vietnamese',
                          'visa',
                          'w2',
                          'wa',
                          'week',
                          'west',
                          'wi',
                          'work',
                          'workday',
                          'worth',
                          'year',
                          'years',
                          'york',
                          'yr',
                          'yrs',
                         '12',
                         'z',
                         '13']

In [ ]:
#left off at travelers

In [51]:
remaining_wordcount = {'cto': 2, 'chief': 12, 'firewall': 9, 'english': 4, 'logging': 1, 'archer': 3, 'generalist': 8, 'midnight': 1, 'telecommunications': 6, 'processing': 3, 'sterling': 11, 'b2bi': 1, 'mac': 6, 'payment': 4, 'reactjs': 23, 'vision': 7, 'vendor': 5, 'datawarehouse': 6, 'iib': 13, 'isso': 4, 'great': 9, 'fte': 7, 'role': 20, 'the': 6, 'east': 2, 'bay!!': 1, 'managed': 9, 'claimcenter': 4, 'appliations': 1, 'sys': 4, 'ladera': 1, 'heights': 3, 'life': 11, 'tri': 1, 'valley': 2, 'company': 8, 'mis': 3, 'video': 16, 'source': 6, 'affiliate': 2, 'specialis': 1, 'disaster': 7, 'bare': 1, 'sox': 4, 'autocad': 3, 'onbase': 6, 'practice': 8, 'bench': 1, 'loan': 4, 'staff': 76, 'winforms': 6, 'elearning': 3, 'date': 1, 'fullstack': 47, 'managing': 3, 'lvl': 2, 'ccie': 3, 'patternmaker': 1, 'fabric': 1, 'finisher': 1, 'ciso': 1, 'passion!': 1, 'labor': 3, 'msbi': 7, 'mvc': 10, 'administror': 1, 'camstar': 1, 'governance': 29, 'robert': 2, 'half': 2, 'polyglot': 1, 'ioc': 1, 'now': 6, 'aml': 5, 'atm': 1, 'eft': 1, 'transaction': 1, 'transactional': 1, 'days': 10, 'wms': 6, 'ultipro': 1, 'comptia': 1, 'a+': 7, 'devloper': 1, 'division': 2, 'communication': 9, 'ediscovery': 2, 'esx': 1, 'an': 3, 'interview': 38, 'jamf': 1, 'hospitality': 1, 'specilaist': 1, 'devopps': 1, 'dynamics365': 1, 'configurations': 2, 'strategist': 16, 'mongo': 5, 'db': 6, 'xamarin': 7, 'absence': 3, 'ai': 12, 'cpq': 5, 'techno': 38, 'privacy': 10, 'atg': 9, 'fortinet': 2, 'focus': 5, 'environment': 13, 'unity': 3, 'game': 5, 'dot': 4, 'multiple': 33, 'positions': 17, 'overnight': 1, 'litigation': 2, 'years': 38, 'memphis': 3, 'wi': 16, 'independent': 6, 'work': 19, 'visa': 5, 'forms': 3, 'german': 1, 'roku': 2, 'editor': 22, 'wordpress': 10, 't': 8, 'experiential': 3, 'voip': 7, 'asic': 12, 'custom': 2, 'jsf': 1, 'financials': 26, 'technological': 1, 'prinicpal': 1, 'trading': 15, 'magento': 8, 'swift': 7, 'casb': 1, 'sourcing': 13, 'nuance': 1, 'shopify': 1, 'expense': 1, 'outside': 2, 'epicor': 4, 'range': 1, 'apis': 2, 'modelizat': 1, 'website': 8, 'hosting': 1, 'e': 20, 'discovery': 2, 'supprot': 1, 'jira': 14, 'ppc': 4, 'seo': 25, 'controlling': 3, 'scm': 22, 'perl': 4, 'osb': 4, 'b2b': 6, 'plains': 6, 'logrhythm': 2, 'opentext': 6, 'captiva': 1, 'scanning': 1, 'vue': 4, 'support"': 1, 'nc"': 1, 'local:': 1, 'dallas': 11, 'defense': 7, 'agency': 7, 'protection': 8, 'gateway': 3, 'upgrade': 6, 'campus': 3, 'aid': 2, 'student': 2, 'hyperion': 16, 'hfm': 3, 'focused': 3, 'ncc': 1, 'rac': 2, 'experienced': 15, 'sailpoint': 14, 'simulation': 1, 'technican': 1, 'audio': 13, 'dsp': 1, 'enthusiastic': 1, 'avaya': 4, 'ip': 6, 'otc': 7, 'le': 5, 'accessory': 1, 'relocation': 2, 'provided': 1, 'packet': 2, 'dir': 7, 'implementing': 1, 'webcenter': 2, 'scada': 6, 'checkpoint': 3, 'varicent': 1, 'etrm': 1, 'claims': 14, 'examiner': 6, 'connectivity': 2, 'intermediate': 7, 'adjuster': 3, 'due': 1, 'diligence': 1, 'selection': 1, 'briefing': 1, 'frontend': 30, 'progammer': 1, 'ivr': 9, 'dover': 1, 'nh': 3, 'seattle': 6, 'wa': 10, 'cincinnati': 4, 'oh': 17, 'qad': 2, 'hive': 3, 'opening': 4, 'framingham': 1, 'san': 26, 'diego': 1, 'ctc': 2, 'philadelphia': 9, 'required:': 4, 'atlanta': 23, 'ga': 33, 'at': 33, 'durham': 2, 'columbus': 8, 'denver': 7, 'co': 35, 'looking': 14, 'nyc': 26, 'option': 8, 'preffered': 1, 'airlines': 1, 'industry': 8, 'permanent': 6, 'immediate': 35, 'oms': 4, 'dodgeville': 1, 'redmond': 2, 'employment': 4, 'appiandeveloper': 1, 'exadata': 9, 'designers': 2, 'hybrid': 19, 'motion': 10, 'merchandiser': 2, 'creative': 26, 'editorial': 4, 'traffic': 18, 'keynote': 1, 'presentation': 14, 'investor': 1, 'relations': 9, 'paid': 17, 'programmatic': 3, 'generation': 2, 'package': 3, 'operational': 5, 'proofreader': 5, 'animator': 2, 'advocacy': 2, 'event': 5, 'interactive': 10, 'art': 21, 'presentations': 1, 'devsigner': 1, 'medi': 1, 'marketaccess': 2, 'hubspot': 1, 'salon': 1, 'campaign': 7, 'partnerships': 2, 'sponsorships': 1, 'archiving': 1, 'sunnyvale': 10, 'austin': 18, 'texas': 7, 'conceptual': 2, 'saas': 17, 'public': 16, 'marcomm': 1, 'collateral': 1, 'pr': 4, 'telecommunication': 4, 'dam': 2, 'image': 2, 'pension': 2, 'planner': 21, 'photo': 2, 'interface': 13, 'poly': 34, 'videographer': 2, 'copy': 9, 'pay': 4, 'per': 3, 'click': 1, 'key': 3, 'programs': 1, 'copyrwriter': 1, 'multimedia': 4, 'sem': 1, 'channels': 2, 'prepayment': 1, 'modeling': 6, 'retoucher': 1, 'photographer': 2, 'membership': 1, 'accounts': 10, 'photography': 1, 'crisis': 1, 'visualization': 13, 'campaigns': 1, 'amazon': 4, 'area': 6, 'fashion': 1, 'coorinator': 1, 'events': 4, 'mm': 28, 'inpatient': 1, 'cerner': 3, 'micro': 9, 'sdk': 1, 'record': 2, 'ads': 2, 'studio': 8, 'ct': 14, 'worker': 1, 'picker': 2, 'packer': 1, 'handler': 3, 'forklift': 2, 'mule': 5, 'any': 3, 'point': 6, 'redux': 4, 'military': 3, 'housing': 1, 'producer': 12, 'multmedia': 1, 'comms': 1, 'accessories': 1, 'pre': 8, 'press': 2, 'demand': 4, 'channel': 3, 'branding': 3, 'mobileiron': 1, 'grail': 1, 'canvas': 1, 'lms': 5, 'networking': 12, 'group': 36, 'community': 4, 'organizer': 1, 'pim': 1, 'strategies': 2, 'snowflake': 8, 'auto': 2, 'itom': 2, 'packaging': 15, 'openings!': 4, 'interior': 4, 'perception': 2, 'apparel': 1, 'optimization': 4, 'ui5': 1, 'mim': 1, 'orders': 1, 'university': 1, 'higher': 3, 'education': 3, 'entertainment': 1, 'automotive': 4, 'opportunities': 2, 'ama': 1, 'wen': 1, 'escalation': 1, 'sprak': 1, 'vertex': 2, 'ncino': 2, 'vfx': 1, 'appv': 1, 'stibo': 2, 'presto': 2, 'solr': 7, 'hands': 1, 'bluecoat': 1, 'proxy': 2, 'treasury': 4, 'successfactor': 1, 'adabas': 3, 'natural': 3, 'biztalk': 5, 'genesys': 6, 'iwd': 1, 'forensics': 2, 'nav': 5, 'institutions': 1, 'advisory': 10, 'f&o': 3, 'tmc': 2, 'gp': 2, 'jee': 4, 'side': 10, 'profit': 3, 'sharing': 3, 'benefit!': 2, 'accept': 1, 'file': 7, 'defined': 2, 'frameworks': 1, 'enablement': 1, 'this': 1, 'is': 24, 'that': 1, 'opportunity!': 2, 'oop': 3, 'chassis': 1, 'brakes': 1, 'analysis': 16, 're': 1, 'come': 1, 'join': 1, 'our': 3, 'beautiful': 1, 'durpal': 1, 'interviewing': 2, 'dulles': 1, 'psft': 1, 'eprocurement': 1, 'practitioner': 6, 'otl': 3, 'module': 5, 'x86': 1, 'price': 1, 'win': 2, 'csso': 1, 'intel': 3, 'provider': 1, 'tbm': 3, 'shaped': 1, 'sqr': 4, 'connected': 4, 'vehicle': 7, 'diagrams': 1, 'topology': 1, 'los': 5, 'angeles': 7, 'ra': 1, 'ppm': 4, 'centered': 1, 'srm': 7, 'scope': 2, 'accelerator': 1, 'special': 2, 'object': 3, 'paired': 1, 'programming': 3, 'builder': 3, 'apache': 5, 'nifi': 1, 'dcl': 1, 'journeyman': 4, 'angularjs': 17, 'driven': 4, 'pentaho': 3, 'hibernate': 3, 'backed': 2, 'doh': 1, 'ndk': 1, 'mcafee': 3, 'testers': 5, 'cellular': 2, 'cdma': 1, 'cmw500': 1, 'lte': 3, 'ethical': 1, 'hacker': 1, 'wm': 16, 'im': 3, 'author': 2, 'itsm': 8, 'smartsheet': 1, 'spotfire': 2, 'r': 13, 'shiny': 3, 'hours': 3, 'week': 2, 'cabling': 2, 'trial': 4, 'attorney': 4, 'orlando': 1, 'idm': 8, 'developmental': 2, 'intern': 15, 'metering': 2, 'id': 3, 'badging': 2, 'compensation': 6, 'law': 2, 'regulation': 1, 'counsel': 1, 'possible': 4, 'lts': 1, 'atsv': 3, 'isoc': 2, 'diagnostic': 1, 'xp': 1, 'fieldglass': 1, 'ft': 14, 'temporary': 2, 'allstate': 1, 'insurance': 20, 'cons': 4, 'estate': 4, 'lease': 2, 'science': 13, 'psychiatrist': 1, 'sec': 2, 'penetration': 7, 'cics': 2, 'jr1005431': 1, 'cold': 1, 'jr1005572': 1, 'jr1003874': 1, 'jr1005574': 1, 'jr1006641': 1, 'lifecycle': 4, 'jr1007279': 1, 'shipping': 7, 'jr1005860': 1, 'jr1005740': 1, 'single': 2, 'ref2411f': 1, 'jr1007475': 1, 'jr1005357': 1, 'ref2252o': 1, 'jr1007278': 1, 'jr1007541': 1, 'jr1007537': 1, 'jr1004514': 1, 'ctw': 1, 'autosys': 1, 'tsang': 1, 'jr1006592': 1, 'spfyjp00000631': 1, 'x3': 1, 'rar': 1, 'css': 6, 'tensorflow': 1, 'bot': 1, 'fill*': 1, 'alfresco': 7, 'exp!': 1, 'comm': 1, 'modelling': 2, 'apaas': 1, 'clojure': 2, 'cssa': 3, 'x': 3, 'case': 2, 'rest': 6, 'lakes': 4, 'line': 2, 'fscm': 8, 'oklahoma': 3, 'city': 18, 'ok': 4, 'pricing': 3, 'webservices': 4, 'advancement': 1, 'space': 5, 'attivio': 1, 'troubleshooting': 1, 'aci': 3, 'f5': 5, 'okc': 1, 'recruitment': 2, 'ga1418': 1, 'detroit': 15, 'mi': 32, 'x2': 2, 'marcom': 1, 'medicare': 2, 'advantage': 1, 'shs': 1, 'diversity': 3, 'supplier': 9, 'analytical': 2, 'loyalty': 1, 'sf': 2, 'firstdocs': 1, 'recognition': 1, 'promotion': 1, 'fluency': 3, 'one': 3, 'connection': 2, 'custody': 2, 'actions': 1, 'gcp': 2, 'plus': 6, 'firstdoc': 1, 'commissions': 2, 'indirect': 1, 'coe': 1, 'excellence': 5, 'national': 3, 'regional': 8, 'severity': 1, 'ap': 3, 'procure': 2, 'configurator': 2, 'temp': 6, 'r+f': 1, 'ppmt': 1, 'incentive': 1, 'floor': 1, 'algo': 1, 'quant': 9, 'hedge': 14, 'fund': 18, 'startup': 26, 'investment': 8, 'elite': 4, 'wealth': 2, 'lock': 1, 'free': 1, 'lockless': 1, 'skills': 4, 'bco': 1, 'seeking': 4, 'elec': 1, 'instrumentation': 4, 'distributed': 8, 'greenfield': 5, 'as': 6, 'plant': 3, 'tivoli': 3, 'tsm': 1, 'spectrum': 2, 'protect': 1, 'ammonia': 4, 'terminal': 7, 'garner': 1, 'disruptive': 2, 'over': 3, 'fpga': 14, 'verilog': 1, 'ultra': 1, 'latency': 2, 'base': 3, 'vb': 9, 'horizon': 1, 'graduates': 1, 'entrepreneurial': 1, 'fixed': 5, 'income': 4, 'high': 6, 'calculations': 1, 'execution': 3, 'california': 2, 'travelers': 1, 'ims': 4, 'supra': 1, 'model': 2, 's2k': 1, 'jvm': 2, 'multithreading': 2, 'complex': 2, 'problem': 4, 'solving': 2, 'fin': 2, 'frankfort': 1, 'quick': 1, 'albany': 2, 'tririga': 6, 'apply': 4, 'techteam': 1, 'welder': 1, 'marklogic': 4, 'fix': 4, 'ritzville': 1, 'broadcast': 1, 'powershell': 4, 'frequency': 1, 'bus': 4, 'perform': 1, 'international': 2, 'stl': 2, 'boost': 2, 'protocols': 1, 'scale': 2, 'platforms': 5, 'capabilities': 1, 'banner': 1, 'bay': 2, 'part': 9, 'maximo': 4, 'locals': 44, 'f2f': 15, 'actuarial': 2, 'mobility': 9, 'ocm': 1, 'works': 2, 'distance': 1, 'readiness': 1, 'cornerstone': 1, 'saba': 1, 'asap': 2, 'bods': 2, 'ipm': 1, 'sharepoint;': 4, 'visio': 6, 'iag': 1, 'cognos': 8, 'puppet': 5, 'scenekit': 1, 'ar': 5, 'vr': 2, 'jd': 3, 'edwards': 3, 'iis': 3, 'mes': 2, 'gxp;': 1, 'vi': 1, 'avoka': 1, 'kcs': 1, 'tickets': 1, 'git': 1, 'ood': 4, 'tdd': 1, 'postgres': 5, 'dxr': 1, 'r3': 1, 'corda': 1, 'blockchain': 11, 'mocha': 1, 'chai': 1, 'administration;': 1, 'scripting;': 1, 'gxp': 1, 'reports': 6, 'forecast': 1, 'biotech': 2, 'pharma': 5, 'suite': 5, 'ifus': 1, 'terraform': 2, 'objective': 1, 'xcode': 1, 'control;': 1, "gmp's;": 1, 'biotech;': 1, 'paas': 1, 'iac': 1, 'pharma;': 1, 'resources': 3, 'hrbp': 1, 'models': 1, 'projects': 7, 'urjent': 1, 'menomonee': 1, 'falls': 1, 'andriod': 2, 'cucumber': 3, 'decommission': 2, 'sa': 3, 'infopath': 1, 'powerpivot': 1, 'oriented': 1, 'telephony': 6, 'vernon': 1, 'hills': 1, 'cd': 8, 'conversion': 9, 'eggplant': 1, 'workflows': 2, 'mssql': 1, 'drug': 1, 'development;': 1, 'biopharma;': 1, 'office;': 1, 'mettawa': 1, 'cards': 2, 'waterfall': 2, 'gap': 1, 'capital': 7, 'urgent': 34, 'rules': 2, 'algorithm': 5, 'pii': 1, 'restful': 2, 'wcf': 3, 'json': 2, 'dns': 3, 'ntp': 1, 'nfs': 1, 'olap': 1, 'uml': 1, 'vba': 1, 'soap': 3, 'strong': 17, 'wcs': 2, 'adms': 1, 'munis': 1, 'acm': 2, 'omaha': 3, 'informix': 1, 'idam': 4, 'sqa': 2, 'infor': 8, 'lawson': 2, 'role!': 4, 'transportation': 5, 'isp': 2, 'nasa': 2, 'iss': 2, 'hvac': 7, 'analytic': 1, 'polygraph': 23, 'ite': 1, 'request': 3, 'fulfillment': 2, 'req': 3, 'feed': 1, 'multi': 9, 'discipline': 1, 'isr': 4, 'pharmacist': 1, 'resident': 3, 'substation': 5, 'hbss': 2, 'metals': 1, 'go': 4, 'iat': 5, 'wti': 1, 'tactical': 1, 'exploitation': 2, 'flight': 4, 'biometrics': 2, 'adjudicator': 2, 'antarctic': 2, 'utilities': 1, 'walpole': 1, 'refrigeration': 2, 'nuclear': 5, 'nonproliferation': 1, 'electronics': 6, 'bliss': 1, 'dhmsm': 1, 'rn': 19, 'cardiac': 2, 'stress': 1, 'secuirty': 1, 'netcool': 2, 'patch': 1, 'metrics': 3, 'backoffice': 1, 'mechatronics': 1, 'patterns': 4, 'mpm': 3, 'satcom': 1, 't2c2': 1, 'c4isr': 1, 'exercise': 4, 'civil': 6, 'p': 2, 'cup': 1, 'utility': 2, 'critical': 3, 'facility': 4, 'uc': 4, 'jrss': 1, 'etch': 2, 'survivability': 1, 'logic': 3, 'pe': 3, 'oil': 5, 'gas': 5, 'load': 5, 'jsc': 1, 'imoc': 1, 'deputy': 1, 'professional': 13, 'army': 1, 'medications': 1, 'integrity': 3, 'markets': 4, 'cnd': 1, 'cognitive': 10, 'warfare': 12, 'lactation': 1, 'weapon': 1, 'fema': 1, 'i:': 1, 'tlm': 1, 'assessor': 2, 'vat': 2, 'cbp': 3, 'pharmacytechnician': 1, 'img': 1, 'presales': 1, 'anywhere': 5, 'u': 1, 'logistic': 1, 'isse': 2, 'hcmc': 1, 'ob': 1, 'machias': 2, 'me': 3, 'atop': 1, 'caregiver': 1, 'riverside': 1, 'sigint': 1, 'air': 3, 'organization': 1, 'ato': 1, 'faa': 2, 'ams': 1, 'wfm': 2, 'nextgen': 2, 'lam': 1, 'vod': 2, 'siem': 3, 'weapons': 2, 'perfusionist': 1, 'essbase': 4, 'springboot': 6, 'elegant': 1, 'mission': 4, 'resolution': 2, 'purchasing': 5, 'wisely': 1, 'cost': 8, 'geologist': 3, 'courier': 1, 'all': 9, 'nursing': 3, 'esoc': 1, 'etime': 2, 'savannah': 1, 'devsecops': 7, 'tmpc': 1, 'aviation': 2, 'weather': 1, 'forecaster': 1, 'return': 4, 'require': 1, 'pivotal': 1, 'foundry': 1, 'pharmacy': 4, 'dh': 1, 'rehab': 1, 'be': 6, 'currently': 1, 'wpoly': 1, 'physician': 3, 'neurology': 2, 'rw': 1, 'semiconductor': 2, 'hot': 5, 'augmented': 1, 'reality': 1, 'start': 2, 'plantation': 1, 'decisioning': 1, 'pvd': 1, 'atc': 1, 'en': 1, 'newport': 1, 'ri': 5, 'waveform': 1, 'consutlant': 2, 'budget': 6, 'anti': 2, 'submarine': 2, 'asw': 2, 'biomedical': 1, 'registered': 1, 'brightspace': 1, 'migrations': 3, 'computational': 4, 'multiphysics': 1, 'cpe': 1, 'turbotax': 1, 'emt': 1, 'paramedic': 1, 'nas': 2, 'grid': 1, 'window': 2, 'solid': 1, 'edge': 1, 'airborne': 2, 'structural': 8, 'ise': 1, 'backbone': 3, 'rdc': 1, 'nagios': 2, 'teamcenter': 8, 'plano': 6, 'thought': 1, 'jdat': 1, 'ci': 11, 'cardiovascular': 2, 'verification': 8, 'globalview': 1, 'lots': 1, 'good': 1, 'deisgner': 1, 'linguist': 2, 'nlp': 3, 'text': 3, 'collections': 5, 'targeter': 3, 'facilitates': 1, 'unified': 8, 'dqe': 1, 'cancer': 1, 'vh': 1, 'infovista': 1, 'institute': 1, 'stability': 2, 'detection': 2, 'investigation': 2, 'tenant': 1, 'improvements': 1, 'mep': 1, 'upgrades': 1, 'specialized': 1, 'ecc': 4, 'bobj': 2, 'fiori': 2, 'required!': 3, 'cos': 1, 'm&s': 1, 'eram': 1, 'pilot': 1, 'tomahawk': 1, 'curriculum': 1, 'investigator': 3, 'appl': 1, 'docsis': 2, 'apo': 9, 'airworthiness': 1, 'offensive': 2, 'teleconference': 1, 'lieutenant': 1, 'spec': 1, 'secure': 1, 'guardian': 1, 'dlp': 3, 'sbc': 1, 'session': 1, 'ac': 1, 'dial': 1, 'portals': 2, 'richfield': 1, 'summer': 1, 'fundamentals': 1, 'boomi': 6, 'underwriter': 4, 'chillers': 1, 'intake': 1, 'hygienist': 1, 'chatbot': 3, 'wpf': 5, 'sfdc': 13, 'vdc': 1, 'buffalo': 1, 'tfs': 3, 'fluid': 3, 'nicu': 1, 'finished': 1, 'vui': 1, 'elimination': 1, 'impact': 2, 'camel': 2, 'redhat': 3, 'sonar': 1, 'outsourcing': 3, 'rating': 1, 'dcgs': 1, 'n': 3, 'ewm': 6, 'backfill': 3, 'stb': 3, 'ave': 1, 'rye': 1, 'micu': 1, 'hospital': 2, 'sourcefire': 1, 'humint': 1, 'quickbooks': 1, 'acceptance': 1, 'signal': 4, 'paper': 1, 'asi': 1, 'ibp': 4, 'clinic': 1, 'tulsa': 1, 'wmd': 1, 'adp': 1, 'ed': 1, 'payable': 3, 'medicine': 1, 'postgresql': 10, 'onclgy': 1, 'based': 11, 'blue': 4, 'earth': 2, 'containerization': 1, 'inver': 1, 'grove': 1, 'orthoptist': 1, 'enforcement': 1, 'phi': 1, 'navy': 3, 'maritime': 1, 'policy': 8, 'appleton': 1, 'minneapolis': 8, 'tm': 3, 'sensor': 1, 'wits': 5, 'phase': 1, 'lambda': 2, 'claim': 2, 'keeping': 1, 'soti': 1, 'zebra': 1, 'lenovo': 1, 'macbooks': 1, 'estimator': 4, 'administrators': 1, 'si': 1, 'bpml': 1, 'fi': 15, 'games': 5, 'vantage': 1, 'bioinformatics': 1, 'chaska': 3, 'html5': 2, 'jll101': 1, 'submission': 1, 'political': 1, 'af': 5, 'caoc': 5, 'lounge': 1, 'qlik': 3, 'sense': 3, 'serialization': 1, 'statistics': 1, 'restaurant': 1, 'tg': 1, 'spv': 2, 'house': 1, 'rx': 1, 'world': 1, 'supt': 1, 'collection': 1, 'mechanic': 2, 'teamsite': 4, 'db2': 12, 'itam': 3, 'script': 3, 'developing': 1, 'fs': 1, 'train': 5, 'safe': 6, 'washington': 1, 'residents': 1, 'protected': 1, 'einstein': 1, 'pdm': 1, 'board': 5, 'circuit': 4, 'occupancy': 1, 'bookkeeper': 3, 'lxp': 1, 'gtims': 1, 'nintex': 1, 'liquidity': 1, 'principle': 4, 'cyberark': 7, 'edifecs': 1, 'developer;': 1, 'hire;': 1, 'horsham': 2, 'discoverant': 1, 'third': 6, 'party': 2, 'repository': 1, 'camera': 2, 'tomography': 1, 'capa': 3, 'skype': 9, 'iq': 3, 'saleforce': 3, 'cleaners': 1, 'symantec': 5, 'xacta': 1, 'share': 3, 'adminstrators': 1, 'psp': 1, 'ec1294': 1, 'workstation': 2, 'incomee': 1, 'methodology': 1, 'engineeer': 1, 'boarding': 1, 'contracting': 2, 'asam': 1, 'continuum': 1, 'chesapeake': 1, 'imagery': 1, 'tool': 4, 'artwork': 1, 'changes': 1, 'timelines': 1, 'underwriting': 2, 'powerplant': 1, 'imanage': 1, 'edocs': 1, 'tracking': 1, 'info': 2, 'eden': 4, 'prairie': 4, 'driver': 4, 'mea': 1, 'furnace': 1, 'gameplay': 2, 'engage': 1, 'adf': 1, 'electric': 1, 'redshift': 4, 'malware': 1, 'actimize': 3, 'entrepreneur': 1, 'sfmc': 1, 'surety': 1, 'assistance': 1, 'kyc': 3, 'vault': 4, 'ps': 10, 'oncology': 4, 'force': 4, 'middle': 2, 'murex': 1, 'synchronization': 1, 'qcp': 1, 'paul': 3, 'qsp': 1, 'mdr': 3, 'remediation': 1, 'schaumburg': 1, 'middleton': 1, 'vrealize': 2, 'technofunctional': 3, 'elasticpath': 2, 'demantra': 1, 'jose': 13, 'concord': 1, 'montvale': 1, 'tv': 1, 'powerbi': 4, 'mandatory': 5, 'wilmington': 3, 'de': 4, 'odi': 3, 'apm': 4, 'upstream': 1, 'nist': 1, 'receptionist': 1, 'plm': 4, 'capture': 3, 'piping': 1, 'survey': 1, 'litium': 1, 'valve': 1, 'consultant~technology': 1, 'audiovisual': 1, 'royalty': 1, 'p2p': 6, 'sustaining': 1, 'longterm': 1, 'contract!!!!!': 1, 'developer~midlevel': 1, 'e&i': 1, 'ore': 1, 'sample': 1, 'prep': 1, 'noise': 1, 'vibration': 1, 'harshness': 1, 'mine': 1, 'dst': 1, 'awd': 1, 'cloudcraze': 2, 'within': 1, 'hana_contract_': 1, 'kalamazoo': 3, 'xml': 4, 'jms': 1, 'ws': 1, 'technologies_contract_san': 1, 'francisco': 3, 'wm\\ewm': 2, 'architect_contract_elyria': 1, 'nodes': 4, 'timer': 1, 'costa': 3, 'mesa': 4, 'ca_contract': 2, 'mdg': 3, 'consultant_contract_kalamazoo': 1, 'snap': 1, 'p2p_contract_foster': 1, 'working': 2, 'integrating': 1, 'ibp_contract_san': 1, 'testing_contract_burbank': 1, 'wm_kalamazoo': 1, 'mi_contract': 1, 'siebel': 4, 'eai': 1, 'xsd': 1, 'xslt': 1, 'transformation_contract_danbury': 1, 'jira_raritan': 1, 'node_raritan': 1, 'nj_c2c': 1, 'consultant_contract_jersey': 1, 'req#': 1, 'sandy': 1, 'ut': 1, 'architect_contract_chicago': 1, 'architect_': 2, 'ca_': 2, 'esb': 2, 'wmb': 3, 'developer_contract_florham': 1, 'park': 2, 'consultant_contract_foster': 1, 'req#:': 1, 'authentication': 1, 'lead_contract_': 1, 'indianapolis': 2, 'spring_contract_charlotte': 1, 'consultant_contract_los': 1, 'northridge': 1, 'fts_contract_': 1, 'developer_costa': 1, 'developer_long': 1, 'term_los': 2, 'developer_contract_charlotte': 1, 'iseries': 3, 'administrator_contract_charlotte': 1, 'architect_contract_king': 1, 'prussia': 2, 'fts_contract_kalamazoo': 1, 'testing_contract_bridgewater': 1, 'blueprism_contract_': 1, 'farmington': 2, 'architect_long': 1, 'experience_contract_elkton': 1, 'md': 14, 'ssas': 2, 'experience_contract_durham': 1, 'cary': 2, 'domain_contract_kalamazoo': 1, 'ewm_contract_san': 1, 'vc': 2, 'sme_contract_elyria': 1, 'ohio': 1, 'sparksql': 1, 'hbase': 1, 'impala': 1, 'hdfs': 1, 'publisher': 1, 'bgn2': 2, 'annuities': 1, 'domain': 11, 'without': 1, 'apian': 1, 'georgia': 15, 'property': 4, 'casualty': 3, 'jmemter': 1, 'company_herndon': 1, 'va_c2c': 1, 'enovia': 1, 'tomorrow': 1, 'interview:': 1, 'through': 1, 'herndon': 3, 'interviews': 23, 'started': 1, 'biomediacal': 1, 'braintree': 1, 'documentum': 7, 'tomorrow:': 1, 'cambridge': 3, 'dveloper': 1, 'eucea': 1, 'bootstrap': 2, 'scheduling': 1, 'consultants': 10, 'sourcer': 3, 'prototyper': 1, 'techincal': 1, 'posting': 1, 'government': 5, 'citizens': 3, 'teams': 1, 'qe': 2, 'techops': 2, 'pythondeveloper': 1, 'xenmobile': 1, 'ab': 3, 'initio': 2, 'openspan': 3, 'arch': 2, 'madison': 7, 'l': 12, 'b': 3, 'hr+exp': 1, 'mii': 4, 'solon': 1, 'lisa': 2, 'michael': 1, 'placeholder': 1, 'gwt': 2, 'callidus': 3, 'rcm': 1, 'ml': 4, 'wecjp00011108': 1, 'ccnp': 2, 'expenses': 3, 'by': 1, 'p&c': 3, 'meridian': 1, 'sast': 1, 'connex': 1, 'onap': 1, 'thunderhead': 1, 'minnetonka': 3, 'billingcenter': 1, 'remedy': 2, 'saint': 2, 'engg': 2, 'bdm': 8, 'locations': 8, 'h1': 2, 'transfers': 3, 'invited': 1, 'k2': 1, 'connect': 4, 'cecl': 1, 'oakland': 1, 'divisional': 1, 'adoption': 2, 'india': 1, 'salary': 3, 'reference': 1, 'green': 4, 'eclipse': 1, 'wifi': 2, 'booking': 1, 'oss': 2, 'specialist_': 1, 'ne': 2, 'hp': 5, 'exstream': 5, 'from': 3, 'home': 7, 'hybris': 13, 'cucm': 1, 'palo': 5, 'alto': 5, 'profiles': 2, 'traveling': 3, 'television': 1, 'confirm': 1, 'slot': 2, 'today': 2, 'was': 5, 'lansing': 4, 'michigan': 3, 'infra': 2, 'who': 1, 'fluent': 1, 'chinese': 1, 'midlevel': 3, 'query': 2, 'whippany': 1, 'emc': 1, 'iec': 3, 'ul60950': 1, 'ul60065': 1, 'culver': 1, 'indiana': 1, 'princeton': 2, 'm204': 2, 'hbits': 1, 'ash': 1, 'experience!!': 1, 'very': 8, 'naperville': 1, 'coldfusion': 4, 'adas': 1, 'feature': 1, 'dimondale': 1, 'rtu': 1, 'prior': 3, 'jquery': 1, 'css3': 1, 'less': 2, 'ajax': 1, 'webstorm': 1, 'sublime': 1, 'photoshop': 2, 'cs5': 4, 'fireworks': 1, 'flex': 1, 'title:': 2, 'relationships': 1, 'institutional': 1, 'clients': 1, 'coaching': 1, 'datastax': 1, 'lync': 2, 'r2a': 1, 'controllership': 1, 'tl': 2, 'sarasota': 1, 'sidney': 1, 'confluence': 1, 'csm': 2, 'developer@$55': 1, 'holmdel': 2, 'bethlehem': 1, 'closer': 1, 'dearborn': 1, 'qliksense': 1, 'processor': 2, 'wcc': 1, 'couchbase': 1, 'sit': 1, 'trader': 2, 'm': 8, 'iprocess': 1, 'jersey': 3, 'temple': 2, 'terrace': 2, 'florida': 4, 'northborough': 1, 'hartford': 2, 'orm': 3, 'sequilize': 1, 'anaylst': 1, 'marlborough': 1, 'annuity': 1, 'uipath': 4, 'hrms': 2, 'should': 1, 'knoxville': 2, 'doors': 1, 'datapower': 7, 'looker': 2, 'requirement': 10, 'forgerock': 2, 'oaks': 2, 'variant': 1, 'g': 7, 'watertown': 1, 'fleet': 2, 'urgently': 2, 'title': 1, 'french': 1, 'speaker': 1, 'mpls': 1, 'ethernet': 1, 'filenet': 4, 'p8': 1, 'jsp': 1, 'servlets': 1, 'empower': 2, 'poweshell': 1, 'malvern': 3, 'metadata': 2, 'governance"': 1, 'cum': 3, 'itbm': 1, 'prod': 2, 'cmdb': 3, 'ramon': 4, 'developer@': 1, 'state': 3, 'h1b': 7, 'sponsorship': 4, 'ann': 2, 'arbor': 2, 'threading': 2, 'payments': 2, 'providers': 1, 'alpharetta': 2, 'face': 7, 'sme_l3': 1, 'lineage': 1, 'troy': 3, 'ec2': 1, 's3': 1, 'dynamodb': 1, 'cdn': 1, 'nginx': 1, 'large': 1, 'templating': 1, 'libraries': 1, 'lightening': 1, 'message': 1, 'sponsership': 1, 'al': 2, 'hitachi': 2, 'conflict': 1, 'ng0624': 1, 'etrading': 1, 'framework': 4, 'dsa': 1, 'written': 1, 'attached': 1, 'blueprism': 2, 'psa': 1, 'databases': 2, 'nice': 1, 'have': 4, 'middlelware': 1, 'page': 1, 'ec': 1, 'ucmdb': 1, 'island': 1, 'intunes': 1, 'liaison': 1, 'nova': 1, 'splicer': 1, 'illinois': 1, 'mapper': 1, 'hadop': 1, 'protocol': 3, 'bethesda': 3, 'fica': 3, 'reopened': 1, 'position:': 1, 'ric': 2, 'telerik': 2, 'kendo': 1, 'norwell': 1, 'mixed': 3, 'writing': 1, 'o': 4, 'notation': 1, 'kotlin': 1, 'eagle': 1, 'manager#': 1, 'eu': 2, 'licensing': 1, 'parsippany': 2, 'offering': 1, 'epm': 8, 'epbcs': 5, 'soft': 3, 'airwatch': 2, 'balancing': 2, 'roles::::multiple': 1, 'path': 5, 'solarwinds': 1, 'sybase': 2, 'airframe': 1, 'factory': 1, 'kitting': 1, 'professionals': 3, 'nor': 1, 'workable': 1, 'edison': 1, 'parties': 4, 'please!!': 2, 'odm': 5, 'louisville': 1, 'ky': 1, 'richardson': 2, 'deerfield': 1, 'massachusetts': 1, 'bothell': 2, 'finacle': 3, 'datacenter': 3, 'reston': 5, 'january': 1, 'virginia': 1, 'lucid': 1, 'dart': 3, 'flutter': 3, 'mountain': 1, 'view': 1, 'saturday': 1, 'manhattan': 2, 'drools': 1, 'location:': 3, 'opt': 6, 'h4': 2, 'spot': 1, 'code': 1, 'cucqrj': 1, 'hire!': 3, 'location': 3, 'mcs': 1, 'pads': 1, 'combustion': 1, 'vsts': 1, 'expertise': 2, 'snow': 2, 'allegro': 1, 'gitlab': 2, 'olean': 1, 'collibra': 3, 'radnor': 2, 'win10': 1, 'pleasanton': 2, 'portland': 2, 'loaniq': 2, 'homeland': 1, 'stored': 1, 'procedure': 1, 'mh2444': 11, 'refresh': 1, 'd*': 1, 'cpt': 6, 'e3': 2, 'holders': 1, 'practitioner_scrum': 1, 'w49452d': 1, 'argo': 1, 'pittsburgh': 6, 'we': 4, 'do': 1, 'provide': 1, 'analyst1': 1, 'sw64201': 1, 'redis': 1, 'hourly': 1, 'maven': 1, 'jenkins': 5, 'f33485d': 1, 'c217d': 1, 'dac': 1, 'headend': 5, 'equities': 4, 'personal': 1, 'ss093': 1, 'person': 19, 'appointment': 1, 'using': 1, 'monica': 1, 'cupid': 1, 'embeeded': 1, 'ssa': 1, 'lsa': 3, 'cle': 1, 'adminstrator': 1, 'furniture': 1, 'gd111': 1, 'hampton': 1, 'greenbelt': 1, 'secops': 1, 'swagger': 2, 'encryption': 3, 'mendix': 1, 'fp&a': 6, 'quote': 1, 'cash': 4, 'taradata': 1, 'candidate': 3, 'dotnet': 2, 'linkedin': 1, 'sfcc': 1, 'charles': 2, 'river': 2, 'medicaid': 3, 'serverless': 2, 'supplies': 1, 'facebook': 1, 'securities': 1, 'classroom': 1, 'layout': 2, 'otm': 2, 'plsql': 2, 'sqlserver': 1, 'debveloper': 1, 'huntersville': 1, 'accepting': 1, 'cots': 1, 'cacs': 1, 'tyson': 1, 'corner': 1, 'hpe': 1, 'aruba': 1, 'memory': 1, 'ic': 1, 'collins': 1, 'fedramp': 1, 'tandem': 3, 'concepts': 1, 'cetified': 1, 'right': 1, 'angle': 1, 'waf': 1, 'rasp': 1, 'moderator': 1, 'mulesfoft': 1, 'word': 1, 'analyst_ne': 1, 'cr': 2, 'duck': 1, 'creek': 1, 'salem': 3, 'wit': 1, 'doamin': 1, 'argus': 2, 'admininstrator': 2, 'i&o': 1, 'warehousedeveloper': 1, 'activiti': 2, 'kubernates': 1, 'irvine': 2, 'bluetooth': 4, 'businesssolutionsarchitect': 1, 'bfsi': 1, 'isv': 1, 'artificial': 1, 'salesforcedeveloperwithlightning': 1, 'ceo': 2, 'ms2': 1, 'coder': 2, 'servant': 1, 'uft': 5, 'cutover': 1, 'streaming': 3, 'jmeter': 1, 'runner': 1, 'networks': 1, 'function': 1, 'issues': 1, 'citizen': 1, 'norwalk': 1, 'afhc': 1, 'interchange': 1, 'implementations': 2, 'author"': 1, 'kinaxis': 2, 'worksoft': 1, 'qas': 1, 'trainee': 1, 's4': 3, 'captital': 1, 'miscrosoft': 1, 'holtsville': 1, 'opportunityl': 1, 'solumnia': 1, 'bhjob2052_15986': 1, 'nd': 2, 'ivanti': 1, 'airport': 1, 'dashboard': 1, 'c++11': 1, 'post': 4, 'remote:': 3, 'bhjob2052_15989': 1, 'programmers': 1, 'waltham': 2, 'king': 1, 'basking': 5, 'ridge': 5, 'berkeley': 1, 'exton': 1, 'woodcliff': 2, 'bhjob2052_15990': 1, 'shift:': 1, 'sat': 1, 'sun': 1, 'wwan': 1, 'katy': 1, 'cfa': 1, 'referral': 1, 'curam': 1, 'eam': 2, 'bhjob2052_15995': 1, 'retirement': 1, 'bhjob2052_16000': 1, 'otmm': 1, 'cpm': 1, 'bhjob2052_16002': 1, 'bhjob2052_15996': 1, 'ppds': 1, 'ehr': 1, 'bhjob2052_16001': 1, 'bhjob2052_15999': 1, 'leading': 3, 'dell': 6, 'rockville': 1, 'blaze': 1, 'carrollton': 2, 'md_12+': 2, 'developer_': 1, 'isu': 4, 'dept': 1, 'developer:': 1, 'not': 1, 'openstack': 1, 'visas': 1, 'successfactors': 5, 'buena': 1, 'papercut': 1, 'printing': 2, 'minimum': 1, 'reliablity': 1, 'ascp': 1, 'scaled': 1, 'developement': 1, 'greenplum': 2, 'pcf': 2, 'cnc': 3, 'jboss': 1, 'rice': 1, 'sciences': 3, 'nx': 1, 't&l': 1, 'behavioral': 1, 'hcm:': 1, 'exp:': 1, 'apptus': 1, 'ion': 1, 'oac': 1, 'sister': 1, 'wmos': 1, 'lotus': 4, 'notes': 3, 'pureconnect': 1, 'sail': 1, 'columbia': 4, 'rocky': 1, 'hill': 1, 'nutanix': 1, 'apple': 1, 'biostatisticians': 1, 'tarrytown': 2, 'bo': 1, 'hoboken': 1, 'jde': 2, 'rhino': 1, 'moq': 1, 'antonio': 2, 'wellmed': 1, 'webmethod': 1, 'room': 2, 'deigner': 1, 'tagging': 2, 'time!': 1, 'company!': 2, 'carlos': 1, 'tadem': 1, 'armonk': 1, 'buckhead': 1, 'fscm&hcm': 1, 'after': 1, 'telephonic': 1, 'charm': 1, 'maintainability': 1, 'category': 3, 'exp_____': 1, 'pci': 2, 'fuse': 1, 'soapui': 1, 'coding': 2, 'senior___________': 1, 'warehousing': 1, 'leadership': 1, 'hrit': 1, 'arm': 1, 'frisco': 1, 'round': 2, 'internet': 2, 'things': 1, 'iiot': 1, 'lombard': 1, 'compute': 1, 'fremont': 1, 'shell': 2, 'ashburn': 1, 'three': 2, 'kensington': 1, 'fhir': 2, 'goldengate': 1, 'developer@philadelphia': 1, 'available': 4, 'facilitator': 1, 'consultant_raritan': 1, 'svc': 1, 'holyoke': 1, 'crd': 2, 'librarian': 1, 'required***': 1, 'exp@philadelphia': 1, 'creo': 1, 'geometric': 1, 'dimensioning': 1, 'tolerancing': 1, 'portsmouth': 2, 'scripts': 2, 'joins': 1, 'testng': 1, 'rensselaer': 1, 'ten': 1, 'vxworks': 1, 'use': 5, 'traditional': 1, 'alternative': 1, 'sl': 1, 'workshop': 1, 'monroe': 1, 'shakopee': 1, 'forest': 1, 'berlin': 1, 'atomsphere': 1, 'vmax': 1, 'powermax': 1, 'locations:': 1, 'identityiq': 1, 'are': 3, 'resources:': 1, 'taleo': 3, 'sfo': 1, 'leandro': 1, 'posdta': 1, 'merchandise': 1, 'extranet': 1, 'slack': 2, 'jaspersoft': 2, 'surveillance': 1, 'coaches': 2, 'juniper': 2, 'riversand': 1, 'opengl': 1, 'premium': 1, 'hunter': 2, 'gl': 4, 'red': 4, 'hat': 3, 'decision': 1, 'r12': 6, 'cxml': 1, 'reader': 1, 'tpm': 1, 'auditing': 1, 'masking': 1, 'as400': 1, 'trackwise': 1, 'ee': 3, 'icm': 1, 'foreign': 1, 'fx': 2, 'dynatrace': 1, 'skilled': 1, 'consolidation': 2, 'roadmapping': 1, 'architectures': 1, 'itsi': 1, 'sto': 1, 'omniture': 1, 'heat': 1, 'adminisrator': 1, 'addison': 1, 'avondale': 1, 'simi': 1, 'boulder': 1, 'bankin': 1, 'butte': 1, 'mt': 1, 'opportunity:': 3, 'pgm': 1, 'usa': 2, 'axiom': 1, 'clara': 1, 'businessobject': 1, 'postgre': 2, 'aam': 1, 'audience': 1, 'anaylyst': 1, 'qsa': 1, 'isa': 1, 'igl': 1, 'client:': 3, 'vancouver': 1, 'wharves': 1, 'edmonton': 1, 'kmcsi': 1, 'om': 1, 'denodo': 1, 'fulltime:': 1, 'guide': 1, 'layers': 1, 'smyrna': 1, 'costpoint': 1, 'two': 1, 'pr_database': 1, 'bentonville': 1, 'junoir': 1, 'chemical': 2, 'modesto': 1, 'schedule': 1, 'develoer': 1, 'permittting': 1, 'pr_senior': 1, 'receiving': 1, 'ids': 3, 'ips': 1, 'merrimack': 2, 'ebusiness:': 1, 'golden': 2, 'gate': 1, 'el': 1, 'segundo': 1, 'qnxt': 1, 'developers@plano': 1, 'abinito': 1, 'oedahc': 1, 'reimbursed': 1, 'sr_': 1, 'union': 1, 'lead_architect': 1, 'veeva': 4, 'fresh': 1, 'undergraduate': 1, 'summit': 1, 'businessobjects': 1, 'arrays': 1, 'fc': 1, 'switches': 1, 'liferay': 4, 'deve': 1, 'rdk': 1, 'please!': 1, 'farmer': 1, 'sector': 1, 'vertical': 1, 'please!!!': 1, 'attend': 1, 'loadrunner': 1, 'payer': 2, 'transactions': 1, 'm3': 1, 'springs': 2, 'lakeland': 2, 'costing': 1, 'extol': 1, 'recent': 1, 'current': 1, 'everett': 1, 'mumps': 1, 'cache': 1, 'sonographer': 1, 'modules': 1, 'receiver': 1, 'warranty': 1, 'qi': 1, 'requirement__mulesoft': 1, 'requirement__cassandra': 1, 'coral': 1, 'grant': 2, 'migrational': 1, 'customers': 1, 'homebase': 2, 'perm:': 1, 'cissp': 1, 'd': 2, 'openshift': 3, 'cc': 2, 'configu': 1, 'analysisexperience': 1, 'mag': 1, 'paralegal': 1, 'functional_remote': 1, 'anuglar': 1, 'cranston': 1, 'developerfull': 1, 'reactor': 1, 'rxm': 1, 'relap': 1, 'hydraulic': 1, 'devtest': 1, 'thingworx': 1, 'lead@atlanta': 1, 'contract:': 1, 'robot': 1, 'urgent:': 2, 'uk': 1, 'freelancer': 1, 'fortune': 2, 'maddy': 20, 'j': 20, 'interoperability': 1, 'esri': 1, 'forsyth': 1, 'optic': 1, 'decatur': 1, 'haven': 1, 'athens': 1, 'hbo': 2, 'fccs': 1, 'etmf': 1, 'gatp': 1, 'vlocity': 2, 'dispatcher': 2, 'supervisory': 1, 'magicscript': 1, 'sage': 1, 'triage': 1, 'immigration': 1, 'unica': 1, 'r&s': 1, 'night': 2, 'haystack': 1, 'llc': 1, 'college': 1, 'grad': 1, 'max!': 1, 'gaming': 3, 'infosphere': 1, 'mortality': 1, 'morbidity': 1, 'bpel': 1, 'partnership': 1, 'zipcar': 1, 'rtl': 1, 'xi': 1, 'layouts': 1, 'animation': 1, 'syracuse': 3, 'wfh': 1, 'jda': 4, 'dms': 1, 'satellite': 2, 'sync': 1, 'superintendent': 3, 'family': 4, 'urgent!': 1, 'rotating': 2, 'adtech': 1, 'b2c': 1, 'giant!': 1, 'doe': 1, 'glassdoor': 1, 'court': 1, 'qualified': 1, 'nm': 1, 'qbrs': 1, 'cno': 1, 'reverse': 1, 'show': 1, 'exhibit': 1, 'raii': 1, 'mainly': 1, 'work!!!': 1, 'pipelines': 1, 'captain': 1, 'graph': 1, 'grants': 3, 'adwords': 2, 'laser': 1, 'crestron': 2, 'npi': 2, 'precast': 1, 'sensors': 1, 'analog': 1, 'multifamily': 1, 'comp': 1, 'depending': 1, 'competitive': 1, 'bim': 1, 'receivable': 3, 'cpa': 1, 'machinist': 2, 'plumbing': 1, 'flexible': 2, 'vision!': 1, 'thin': 1, 'film': 1, 'coating': 1, 'chambers': 1, 'energy': 1, 'silicon': 2, 'photonics': 1, 'willing': 1, 'buildings': 2, 'mems': 1, 'cloudops': 1, 'physical': 1, 'early': 1, 'sports': 1, 'athletics': 1, 'relocate!': 1, 'socket': 1, 'excellent': 1, 'benefits!': 1, 'vpn': 1, 'growth!': 1, 'tcp': 1, 'senior;': 1, 'massive': 1, 'employee': 2, 'stock': 1, 'investigative': 1, 'rme': 1, 'dt': 1, 'la': 1, 'electromechanical': 1, 'rigger': 1, 'sprinkler': 1, 'sorting': 1, 'oshpd': 1, 'graphql': 1, 'revit': 1, 'vuejs': 1, 'modernization': 1, 'lmp': 1, 'sacm': 3, 'vgtn001': 1, 'idp': 2, 'ledger': 1, 'canada': 1, 'webservice': 1, 'janitor': 1, 'marshall': 1, 'a&p': 1, 'licensed': 1, 'cekms': 1, 'mifflin': 1, 'arabic': 1, 'consecutive': 1, 'schenectady': 1, 'govt': 1, 'pbm': 1, 'rtm': 1, 'h': 1, 'actuary': 1, 'bss': 1, 'roadway': 1, 'highway': 1, 'paris': 1, 'alma': 1, 'workflow': 2, 'geoscience': 1, 'castle': 1, 'rock': 1, 'l4': 1, 'wv!!!': 1, 'oab': 1, 'budgeting': 1, 'spb': 1, 'mathematician': 1, 'cryptanalytic': 1, 'mathematical': 2, 'ax092': 1, 'gs': 1, 'attack': 1, 'responder': 1, 'lpb': 1, 'starlims': 1, 'rts': 2, 'f': 4, 'rulestream': 1, 'radiant': 1, 'closing': 1, 'amica': 1, 'tomball': 1, 'programme': 1, 'methods': 1, 'spc': 1, 'moogsoft': 1, 'aiops': 1, 'momentum': 1, 'o&m': 2, 'birt': 1, 'xpression': 1, 'adviser': 1, 'upw': 1, 'waste': 1, 'water': 2, 'cmp': 1, 'sale': 1, 'oem': 1, 'microservice': 1, 'telematics': 1, 'symphony': 1, 'move': 1, 'mergers': 1, 'acquisitions': 1, 'floater': 1, 'sysadmin': 1, 'rsa': 1, 'journey': 1, 'food': 1, 'printed': 1, 'chip': 1, 'programers': 1, 'ln': 2, 'expansion': 2, 'learn': 3, 'autonomous': 4, 'bonuses': 1, 'saving': 2, 'crypto': 1, 'deep': 1, 'profitable': 1, 'aka': 1, 'sdn': 1, 'rust': 1, 'regeneron': 3, 'genetics': 3, 'genomics': 1, 'eq': 1, 'geographic': 1, 'gatos': 1, 'vcloud': 1, 'scrummaster': 1, 'sustenance': 1, 'compellent': 2, 'scpo': 1, 'similar': 2, "req'd": 4, 'intactix': 2, 'lotusscript': 1, 'jdewards': 1, 'enterpriseone': 1, 'drafters': 1, 'tms': 1, 'iaas': 1, 'engineer__': 1, 'newark': 1, 'salesforcedx': 1, 'sfdx': 1, 'vim': 1, 'brms': 1, 'stamford': 1, 'heroku': 1, 'cmc': 1, 'foster': 1, 'license': 1, 'cygnet': 1, 'meanstack': 1, 'ueba': 1, 'exabeam': 1, 'vb6': 2, 'businessworks': 1, 'ver': 1, 'some': 2, '>>sr': 1, '>>san': 1, 'pbcs': 1, 'eto': 1, 'partially': 1, 'gts': 1, 'sip': 1, 'di': 1, 'ss': 2, 'oshkosh': 1, 'beanshell': 1, 'ms365': 1, 'sdwan': 1, 'county': 1, 'largo': 1, 'c+': 1, 'hpc': 1, 'gemfire': 1, 'pmi': 1, 'batch': 2, 'brea': 2, 'fm': 5, 'gm': 4, 'funds': 1, 'westlake': 1, 'scottsdale': 2, 'develoepr': 1, 'jitterbit': 1, 'switching': 1, 'duluth': 1, 'tridion': 2, 'hw': 1, 'ground': 1, 'enginee': 1, 'orchestration': 1, 'just': 4, 'in!': 3, 'in!!!': 1, 'roles!': 2, 'position!': 2, 'developers!': 1, 'firm!': 1, 'department': 1, 'new!!!': 1, 'dunwoody': 1, 'osisoft': 1, 'knockout': 1, 'petrel': 2, 'transformational': 1, 'tad': 1, 'also': 3, 'can': 3, 'docs': 1, 'solutionit': 1, 'managementmanager': 1, 'reportum': 1, 'policycenter': 2, 'ca&': 1, 'opt&': 1, 'minnesota': 1, 'econsent': 1, 'crud': 1, 'scenarios': 1, 'reinsurance': 1, 'xs': 1, 'bizid#6157': 1, 'cae': 1, 'dx': 2, 'identify': 1, 'ordinator': 1, 'va____sg': 1, 'inperson': 3, 'hitrust': 1, 'cleveland': 1, 'temperance': 1, 'melville': 1, 'des': 1, 'moines': 1, 'fargo': 1, 'daytona': 2, 'kegonsa': 1, 'hipaa': 1, 'westbury': 1, 'harrisburg': 1, 'brf+': 1, 'burbank': 2, 'writer:': 1, 'extjs': 1, 'rth': 1, 'idms': 1, 'cis': 1, 'isim': 1, 'ptp': 1, 'wi!': 1, 'subsurface': 1, 'qliksens': 1, 'nyhx': 2, 'dhs': 2, 'forescout': 1, 'epa': 3, 'usda': 1, 'ncis': 1, 'cartographer': 1, 'schedulers': 1, 'nih': 1, 'machinery': 1, 'forecasting': 1, 'converged': 1, 'eastern': 1, 'states': 1, 'rq24428': 1, 'fdic': 1, 'requirement::': 1, 'profile': 1, 'prototype': 1, 'hiring::': 1, 'clsa': 1, "visa's": 1, 'rq35690': 1, 'homldel': 1, 'bell': 1, 'owings': 1, 'mills': 1, 'pipeline': 1, 'redwood': 1, 'angular__kp': 1, 'apic': 1, 'certifications': 1, 'required!!': 1, 'slo': 1, 'salesforcedeveloper': 1, 'liberty': 1, 'newtown': 1, 'square': 1, 'purchase': 1, 'better': 1, 'implementor': 1, 'roll': 1, 'out': 1, 'kpi': 1, 'partners': 1, 'openair': 1, 'sustainment': 1, 'cubes': 1, 'capps': 1, 'wih': 1, 'sdl': 1, 'toad': 1, 'middletier': 1, 'trenton': 1, 'tanium': 1, 'stealthwatch': 1, 'hippa': 1, 'fresher': 1, 'role:': 1, 'sophos': 1, 'bussiness': 1, 'school': 1, 'depot': 1, 'fast': 1, 'growing': 1, 'startup!': 1, 'hq': 1, 'octo': 1, 'prefer': 1, 'qt': 1, 'fluentd': 1}

In [52]:
remaining_wordcount

{'cto': 2,
 'chief': 12,
 'firewall': 9,
 'english': 4,
 'logging': 1,
 'archer': 3,
 'generalist': 8,
 'midnight': 1,
 'telecommunications': 6,
 'processing': 3,
 'sterling': 11,
 'b2bi': 1,
 'mac': 6,
 'payment': 4,
 'reactjs': 23,
 'vision': 7,
 'vendor': 5,
 'datawarehouse': 6,
 'iib': 13,
 'isso': 4,
 'great': 9,
 'fte': 7,
 'role': 20,
 'the': 6,
 'east': 2,
 'bay!!': 1,
 'managed': 9,
 'claimcenter': 4,
 'appliations': 1,
 'sys': 4,
 'ladera': 1,
 'heights': 3,
 'life': 11,
 'tri': 1,
 'valley': 2,
 'company': 8,
 'mis': 3,
 'video': 16,
 'source': 6,
 'affiliate': 2,
 'specialis': 1,
 'disaster': 7,
 'bare': 1,
 'sox': 4,
 'autocad': 3,
 'onbase': 6,
 'practice': 8,
 'bench': 1,
 'loan': 4,
 'staff': 76,
 'winforms': 6,
 'elearning': 3,
 'date': 1,
 'fullstack': 47,
 'managing': 3,
 'lvl': 2,
 'ccie': 3,
 'patternmaker': 1,
 'fabric': 1,
 'finisher': 1,
 'ciso': 1,
 'passion!': 1,
 'labor': 3,
 'msbi': 7,
 'mvc': 10,
 'administror': 1,
 'camstar': 1,
 'governance': 29,
 'robert

In [24]:
#Sort Word Count
#return a sorted word count list
def sortWordCount(wordCountDict):
    return sorted(wordCountDict, key = wordCountDict.get, reverse=True)

In [25]:
word_count_dict = sortWordCount(word_count)

---

In [ ]:
#Based on Given Percentage, output list of top match word
def getHighFrequencyWords(wordCountDict, sortedWordCountList, percentage=0.6):
    count = 0;
    total = sum(wordCountDict.values())
    expected_count = total * percentage
    index = 0
    for word in sortedWordCountList:
        count += wordCountDict[word]
        if count > expected_count:
            index = sortedWordCountList.index(word)
            break
    result = [s for s in sortedWordCountList[0:index] if s != len(s) * s[0]]
    result = [s for s in result if "NN" in nltk.pos_tag([s])[0][1]]
    result = [s for s in result if s != "Sr"]
    return result


In [ ]:
high_freq = getHighFrequencyWords(word_count, word_count_dict)

In [ ]:
#Filter Words from raw Job Title use the high frequency word we have built
def filterWordsFormRawJobTitle(jobtitle_list, high_frequency_list):
    jobtitle_list =  [" ". join(filter(lambda x: x in high_frequency_list, title.split())) for title in jobtitle_list]
    jobtitle_list =  [jobtitle for jobtitle in jobtitle_list if len(jobtitle.split()) >= 2]
    return jobtitle_list


In [ ]:
filtered_words = filterWordsFormRawJobTitle(job_list, high_freq)

In [ ]:
#def unique_list(l):
 #   ulist = []
  #  p = inflect.engine()
   # [ulist.append(x) for x in l if x not in ulist]
    #ulist = [p.singular_noun(x) if p.singular_noun(x) else x for x in ulist]
    #return ulist


In [ ]:
#unique_job_list = unique_list(filtered_words)

In [ ]:
#A bunch of string normalization 
def stringNormalization(jobtitle_list):
    #Change To Camel Case
    jobtitle_list = [' '.join(word[0].upper() + word[1:] for word in job.split()) for job in jobtitle_list]
    #Remove Duplicate Words
    #jobtitle_list = [" ".join(unique_list(job.split())) for job in jobtitle_list]
    #Aggregate to Set
    return list(set(jobtitle_list))

In [ ]:
normalized = stringNormalization(filtered_words)

In [ ]:
len(normalized)

In [ ]:
normalized

---

In [ ]:
import argparse
import re
import nltk
import sys
import inflect

#Revmoe Punctuations that definitely won't contribute
def remove_punctuation(s):
    #Phase 1, handle punctuation.
    # Directly removable
    #  '/', ',', '\', '|',
    for char in ['/', '//', '|', '?', '(', ')', '{', '}', '[', ']', ',', '.', '-']:
        s = s.replace(char, " ")
    s.strip()
    #remove those consists of only punctuation and number
    s = filter(lambda x: not re.match("[0-9~!@#$%^&*()_\-+{}\":;\']+", x), s.split())
    return ' '.join(s)

In [ ]:
def removePunctuation(jobtitle_list):
    return [remove_punctuation(s) for s in jobtitle_list]


In [ ]:
def wordCount(jobtitle_list):
    wordcount = {}
    for jobtitle in jobtitle_list:
        for word in jobtitle.split():
            if word not in wordcount:
                wordcount[word] = 1
            else:
                wordcount[word] += 1
    return wordcount

In [ ]:
def sortWordCount(wordCountDict):
    return sorted(wordCountDict, key = wordCountDict.get, reverse=True)

In [ ]:
def getHighFrequencyWords(wordCountDict, sortedWordCountList, percentage=0.6):
    count = 0;
    total = sum(wordCountDict.values())
    expected_count = total * percentage
    index = 0
    for word in sortedWordCountList:
        count += wordCountDict[word]
        if count > expected_count:
            index = sortedWordCountList.index(word)
            break
    result = [s for s in sortedWordCountList[0:index] if s != len(s) * s[0]]
    result = [s for s in result if "NN" in nltk.pos_tag([s])[0][1]]
    result = [s for s in result if s != "Sr"]
    return result


In [ ]:
def filterWordsFormRawJobTitle(jobtitle_list, high_frequency_list):
    jobtitle_list =  [" ". join(filter(lambda x: x in high_frequency_list, title.split())) for title in jobtitle_list]
    jobtitle_list =  [jobtitle for jobtitle in jobtitle_list if len(jobtitle.split()) >= 2]
    return jobtitle_list


In [ ]:
def stringNormalization(jobtitle_list):
    #Change To Camel Case
    jobtitle_list = [' '.join(word[0].upper() + word[1:] for word in job.split()) for job in jobtitle_list]
    #Remove Duplicate Words
    jobtitle_list = [" ".join(unique_list(job.split())) for job in jobtitle_list]
    #Aggregate to Set
    return list(set(jobtitle_list))